# Import

In [1]:
import matplotlib.ticker as plticker
import cv2
import pytesseract
from PIL import Image
import os
import numpy as np
import pandas as pd
from dbr import DynamsoftBarcodeReader
from matplotlib import pyplot as plt
from skimage.measure import compare_ssim
import imutils
import statistics
from scipy import stats
import random
from difflib import SequenceMatcher
from IPython.display import display, Markdown, Latex
%matplotlib inline

# FILES

In [2]:
os.chdir('/Users/kunal/Documents/VdartWorking/realFake/RunningPresent/')

## TESTING IMAGE

In [3]:
MAINIMAGEFILEPNG = '/Users/kunal/Documents/VdartWorking/SampleCopies/GenuineCopies/H1B - Tushar Sharad Part 1PNG.PNG'
MAINPDFFILE = '/Users/kunal/Documents/VdartWorking/SampleCopies/GenuineCopies/H1B - Tushar Sharad Part 1.pdf'

In [4]:
CLEANARRAYFORMAINIMAGEFILEPNG = cv2.imread(MAINIMAGEFILEPNG)
x11, y11, z11 = CLEANARRAYFORMAINIMAGEFILEPNG.shape
CLEANARRAYFORMAINIMAGEFILEPNG = cv2.resize(CLEANARRAYFORMAINIMAGEFILEPNG, dsize=(698, 910), interpolation=cv2.INTER_CUBIC)

## REAL Image

In [5]:
COMPAREIMAGEFILEPNG = '/Users/kunal/Documents/VdartWorking/realFake/Document.jpg'

In [6]:
CLEANARRAYFORCOMPAREIMAGEFILEPNG = cv2.imread(COMPAREIMAGEFILEPNG) 
x2, y2, z2 = CLEANARRAYFORCOMPAREIMAGEFILEPNG.shape
CLEANARRAYFORCOMPAREIMAGEFILEPNG = cv2.resize(CLEANARRAYFORCOMPAREIMAGEFILEPNG, dsize=(698, 910), interpolation=cv2.INTER_CUBIC)

# Display the Image

In [7]:
imageCV = CLEANARRAYFORMAINIMAGEFILEPNG

# Barcode

In [8]:
def checkEqual1(iterator):
    iterator = iter(iterator)
    try:
        first = next(iterator)
    except StopIteration:
        return True
    return all(first == rest for rest in iterator)

In [9]:
def getText(result, partText):
    endindex = result + 80
    partialdetected = text[result:endindex]
    endindex = partialdetected.find("\n")
    fulldetected = partialdetected[:endindex]
    startindex = fulldetected.find(partText)#'er'
    barcodeNumberdetectedname = fulldetected[startindex+3:28]
    return barcodeNumberdetectedname

## Detection

In [10]:
#t0068MgAAACk+RNudXWjI8IqSZ6cPB04DUpOUhqTHb7YTFYTgM+nuTsx92c7RDeCS6o417Oo6MKjRUn0LnCMqfxz8JHqN74w=

In [11]:
barcodeArray = []
dbr = DynamsoftBarcodeReader()
dbr.initLicense('t0068MgAAACk+RNudXWjI8IqSZ6cPB04DUpOUhqTHb7YTFYTgM+nuTsx92c7RDeCS6o417Oo6MKjRUn0LnCMqfxz8JHqN74w=') # https://www.dynamsoft.com/CustomerPortal/Portal/Triallicense.aspx
results = dbr.DecodeFile(MAINPDFFILE)
textResults = results["TextResults"]
for textResult in textResults:
    #print(textResult["BarcodeText"])
    barcodeArray.append(textResult["BarcodeText"])

In [12]:
if len(barcodeArray) == 0:
    #print("NO BARCODE DETECTED!!!!")
    barcodeDetected = "N/A"
elif len(barcodeArray) == 1:
    barcodeDetected = barcodeArray[0]
elif len(barcodeArray) > 1:
    working = checkEqual1(barcodeArray)
    if working:
        barcodeDetected = barcodeArray[0]
    else:
        barcodeDetected = "N/A"

## Tesseract Compare

In [13]:
OCRPIC = MAINIMAGEFILEPNG
value=Image.open(OCRPIC)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe"
text = pytesseract.image_to_string(value, config='')

In [14]:
result = text.find('Receipt#') 
result2pt1  = text.find('Receipt Number ')
textpt1 = text[result2pt1:len(text)]
result2 = textpt1.find('Receipt Number ')
result1Final = getText(result,'pt')
result2Final = getText(result2, 'er')
if result1Final.endswith(";"):
    result1Final = result1Final[:-1]
if result2Final.endswith(";"):
    result2Final = result2Final[:-1]
numSpace1 = result1Final.count(" ")
numSpace2 = result2Final.count(" ")
if numSpace1 > 1:
    positionofthing = result1Final.find(" ")
    if not positionofthing == -1:
        result1Final = result1Final[0:positionofthing]    
if numSpace2 > 1:
    positionofthing2 = result2Final.find(" ")
    if not positionofthing2 == -1:
        result2Final = result2Final[0:positionofthing2]

result1Final = result1Final.strip()
result2Final = result2Final.strip()

In [16]:
equal = False
if result1Final == result2Final:
    if result1Final == barcodeDetected:
        equal = True
    elif result1Final != barcodeDetected:
        equal = False
if len(result1Final) < 5 and len(result2Final) < 5:
    equal = False
if len(result1Final) < 5:
    if result2Final == barcodeDetected:
        equal = True
    else:
        equal = False
if len(result2Final) < 5:
    if result1Final == barcodeDetected:
        equal = True
    else:
        equal = False

In [17]:
BARCODEISCORRECT = equal

# DATE DETECTION

In [18]:
result = text.find('Valid from') 
big = text[result:result+35]

In [19]:
start = big.find("from")
startstr = big[start+5:start+15]
end = big.find("to")
endstr = big[end+3:end+13]

In [ ]:
start = startstr.split("/")
end = endstr.split("/")

In [ ]:
work = False
reason = ""
if ((len(start) == 3) and (len(end) == 3)):    
    if(start[2] > end[2]):
        work = False
        reason = "Year is before the starting one"
    if(start[2] == end[2]):
        if(end[0] < start[0]):
            work = False
            reason = "Month Doesn't Work"
        if(end[0] > start[0]):
            work = True
            reason = "Month Works"
        if(end[0] == start[0]):
            if(end[1] < start[1]):
                work = False
                reason = "Day Doesn't Work"
            if(end[1] > start[1]):
                work = True
                reason = "Day Works"
            if(end[1] == start[1]):
                work = False
                reason = "Date Same"
    if(start[2] < end[2]):
        work = True
        reason = "Year Works"
    if(int(start[1]) > 31 or int(end[1]) > 31 or int(start[0]) > 12 or int(end[0]) > 12):
        work = False
        reason = "Date out of bounds"
else:
    work = False
    reason = "The array is wrong"

## Ouput

In [ ]:
DATEWORKING = work
WHYDATEISWORKING = reason
#print("The Date ", work, " becuase ", reason)

# Title Compare

## Testing Images

In [ ]:
titleimgCV = CLEANARRAYFORMAINIMAGEFILEPNG[0:50, 10:650]

In [ ]:
titleimgCV2= CLEANARRAYFORCOMPAREIMAGEFILEPNG[0:50, 10:650]

In [ ]:
original = titleimgCV2 #know real
duplicate = titleimgCV #dont know

In [ ]:
threshold = 50
TITLEIMAGECOMPARE = False

In [ ]:
if original.shape == duplicate.shape:
    difference = cv2.subtract(original, duplicate)
    b, g, r = cv2.split(difference)

In [ ]:
x,y = b.shape
size = x*y
countb = 0
for startthing1 in b:
    for endthomg1 in startthing1:
        if endthomg1 > 50:
            countb+=1
countg = 0
for startthing2 in g:
    for endthomg2 in startthing2:
        if endthomg2 > 50:
            countg+=1
countr = 0
for startthing3 in r:
    for endthomg3 in startthing3:
        if endthomg3 > 50:
            countr+=1
PERCENT_BLUE_DIFFERENCE_TITLE = countb/size*100
PERCENT_GREEN_DIFFERENCE_TITLE = countg/size*100
PERCENT_RED_DIFFERENCE_TITLE = countr/size*100

In [ ]:
if PERCENT_BLUE_DIFFERENCE_TITLE < 15 and PERCENT_GREEN_DIFFERENCE_TITLE < 15 and PERCENT_RED_DIFFERENCE_TITLE < 15:
    TITLEIMAGECOMPARE = True

# Text

In [ ]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
OCRPIC = COMPAREIMAGEFILEPNG
value=Image.open(OCRPIC)
text2 = pytesseract.image_to_string(value, config='')

In [ ]:
yes = 0
no = 0
totoal = 0
for i in range(100):
    startingValueofSTring = random.randint(1,len(text2))
    lengthofString = random.randint(3,10)
    small_string = text2[startingValueofSTring:startingValueofSTring+lengthofString]
    has_small_string = False
    if small_string in text:
        has_small_string = True
    if has_small_string:
        yes+=1
    else:
        no+=1
    totoal+=1
diffsimiliar = yes/totoal
diffdifferent = no/totoal
similiar = similar(text, text2)

## Output

In [ ]:
if (similiar < 0.35):
    SIMILARTYBETWEENTEXTINDOCUMENT = False
else:
    SIMILARTYBETWEENTEXTINDOCUMENT = True

# WaterMark

## Basic Removal of Real IMAGE

In [ ]:
down2 = 239
up2 = 248 
array = CLEANARRAYFORMAINIMAGEFILEPNG
arrayNew = CLEANARRAYFORMAINIMAGEFILEPNG
w, h, j = array.shape
array2 = CLEANARRAYFORCOMPAREIMAGEFILEPNG
arrayNew2 = CLEANARRAYFORCOMPAREIMAGEFILEPNG
w2, h2, j2 = array2.shape
falsepixel = 0
correctpixel = 0
errorpixel = 0
totalpixel = 0
for x2 in range(w2-1):
    for y2 in range(h2-1):
        singlearray2 = array2[x2, y2]
        r2 = singlearray2[0]
        g2 = singlearray2[1]
        b2 = singlearray2[2]
        if (r2 > down2 and r2 < up2 and g2 > down2 and g2 < up2 and b2 > down2 and b2 < up2):
            arrayNew2[x2, y2] = [0, 255, 0]
            singlearray = array[x2,y2]
            r = singlearray[0]
            g = singlearray[1]
            b = singlearray[2]
            if (r == 0 and g == 0 and b == 0):
                falsepixel+=1
            elif (r == 0 and g == 255 and b == 0):
                correctpixel +=1
            else:
                errorpixel += 1
            totalpixel+=1
        else:
            arrayNew2[x2, y2] = [0, 0, 0]
img2final = Image.fromarray(arrayNew2)

In [ ]:
im1 = img2final.save("REALImageWaterMark.jpg")

## FINAL COMPARE OF WATERMARK

In [ ]:
down = 239#238 ----- 239 
up = 248 # ---------248
array = CLEANARRAYFORMAINIMAGEFILEPNG
array2 = CLEANARRAYFORCOMPAREIMAGEFILEPNG
arrayNew = CLEANARRAYFORMAINIMAGEFILEPNG
arrayNew2 = CLEANARRAYFORCOMPAREIMAGEFILEPNG
w,h,c = array.shape
w2,h2,c2 = array2.shape
NUMOFPIXELSHOULDTHEREBUTNOT = 0
NUMOFPIXELNOTBUTTHERE = 0
NUMOFCORRECTPIXEL = 0
totalCOUNT = 0
for x in range(w-1):
    for y in range(h-1):
        singlearray = array[x, y]
        r = singlearray[0]
        g = singlearray[1]
        b = singlearray[2]
        if (r > down and r < up and g > down and g < up and b > down and b < up):
            definitivVALUE = True
            totalCOUNT+=1
            arrayNew[x, y] = [0, 255, 0]
        else:
            definitivVALUE = False
            arrayNew[x, y] = [0, 0, 0]
        singlearray2 = array2[x, y]
        r2 = singlearray2[0]
        g2 = singlearray2[1]
        b2 = singlearray2[2]
        if (r2 > down and r2 < up and g2 > down and g2 < up and b2 > down and b2 < up):
            pixelDONTKNOW = True
        else:
            pixelDONTKNOW = False
        if(definitivVALUE and pixelDONTKNOW):
            #WHEN PIXEL SHOULD BE IN RANGE AND IS IN RANGE
            NUMOFCORRECTPIXEL+=1
        elif(definitivVALUE and not pixelDONTKNOW):
            #WHEN PIXEL SHOULD BE IN RANGE BUT ISNT
            NUMOFPIXELSHOULDTHEREBUTNOT+=1
        elif(not definitivVALUE and definitivVALUE):
            #WHEN PIXEL SHOULDN'T BE IN RANGE BUT IS
            NUMOFPIXELNOTBUTTHERE+=1      
img = Image.fromarray(arrayNew)

## Compare Value Around Pixel

## Final Compare Around PIXEL WITH DISTANCE

In [ ]:
file1 = 'REALImageWaterMark.jpg'

In [ ]:
row = []
column = []
array = cv2.imread(file1)
x_length = len(array[0])
y_length = len(array)
for i in range(x_length):
    row.append(str(i))
for i in range(y_length):
    column.append(str(i))
df = pd.DataFrame(index=row, columns=column)

In [ ]:
array = cv2.imread(file1)
arrayNew = cv2.imread(file1)
NUMOFPIXELSHOULDTHEREBUTNOTAVERAGE = 0
NUMOFPIXELNOTBUTTHEREAVERAGE = 0
NUMOFCORRECTPIXELAVERAGE = 0
totalCOUNTAVERAGE = 0
definitivVALUEAVERAGE = False
distance = 2
dis = distance
for thingthing in range(len(array[0])-distance):
    for thingthingYY in range(len(array)-distance):
        center_XXX = thingthing
        center_YYY = thingthingYY
        distance = dis
        partArrDis = [[0 for x in range(distance*2+1)] for y in range(distance*2+1)]
        center_XX = center_XXX
        center_YY = center_YYY
        for uu in range(distance*2+1):
            for rr in range(distance*2+1):
                a,b,c = array[uu+(thingthingYY-distance*2+1)][rr+(thingthing-distance*2+1)]
                theSingleArrayABC = [a,b,c]
                partArrDis[uu][rr] = theSingleArrayABC
        countGreen = 0
        countBlack = 0
        totalCount = 0
        theXofARRRAY = len(partArrDis)
        theYofARRRAY = len(partArrDis[0])
        for aa in range(theXofARRRAY):
            for bb in range(theYofARRRAY):
                singleARRAYwPART = partArrDis[aa][bb]
                rr, gg, bb = singleARRAYwPART
                if rr < 30 and gg < 30 and bb < 30:
                    countBlack += 1
                else:
                    countGreen += 1    
                totalCount+=1
        perceBlack = (countBlack/totalCount) * 100
        perceGreen = (countGreen/totalCount) * 100
        x = perceBlack+perceGreen
        IDK_X = center_XXX-distance*2+1
        IDK_Y = center_YYY-distance*2+1
        if (perceBlack > 40):
            arrayNew[IDK_Y][IDK_X]=[0,0,0]
            definitivVALUEAVERAGE = False
        else:
            arrayNew[IDK_Y][IDK_X] = [0,255,0]
            definitivVALUEAVERAGE = True
        singlearray2 = array2[thingthingYY, thingthing]
        r2 = singlearray2[0]
        g2 = singlearray2[1]
        b2 = singlearray2[2]
        if (r2 > down and r2 < up and g2 > down and g2 < up and b2 > down and b2 < up):
            pixelDONTKNOWAVERAGE = True
        else:
            pixelDONTKNOWAVERAGE = False
        if(definitivVALUEAVERAGE and pixelDONTKNOWAVERAGE):
            #WHEN PIXEL SHOULD BE IN RANGE AND IS IN RANGE
            NUMOFCORRECTPIXELAVERAGE+=1
        elif(definitivVALUEAVERAGE and not pixelDONTKNOWAVERAGE):
            #WHEN PIXEL SHOULD BE IN RANGE BUT ISNT
            NUMOFPIXELSHOULDTHEREBUTNOTAVERAGE+=1
        elif(not definitivVALUEAVERAGE and definitivVALUEAVERAGE):
            #WHEN PIXEL SHOULDN'T BE IN RANGE BUT IS
            NUMOFPIXELNOTBUTTHEREAVERAGE+=1     
        totalCOUNTAVERAGE+=1

In [ ]:
img = Image.fromarray(arrayNew)
im1 = img.save("GroupedPixeledIMAGE.jpg")

## Output

In [ ]:
WATERMARKOUTPUTFINAL=False
WATERMARKOUTPUTave=False
WATERMARKOUTPUT2=False
WATERMARKOUTPUTFINAL=False
coorect = correctpixel/totalpixel*100
worng = falsepixel/totalpixel*100
error = errorpixel

aveCorrect = NUMOFCORRECTPIXELAVERAGE/totalCOUNTAVERAGE *100
aveWRONG = NUMOFPIXELSHOULDTHEREBUTNOTAVERAGE/totalCOUNTAVERAGE *100
aveERROR = NUMOFPIXELNOTBUTTHEREAVERAGE

correctPT2 = NUMOFCORRECTPIXEL/totalCOUNT *100
wrongPT2= NUMOFPIXELSHOULDTHEREBUTNOT/totalCOUNT*100
errorPT2 = NUMOFPIXELNOTBUTTHERE
if coorect < 10:
    WATERMARKOUTPUT1 = False
else:
    WATERMARKOUTPUT1 = True
if aveCorrect < 10:
    WATERMARKOUTPUTave = False
else:
    WATERMARKOUTPUTave = True
if correctPT2 < 10:
    WATERMARKOUTPUT2 = False
else:
    WATERMARKOUTPUT2 = True

if WATERMARKOUTPUT1 == True and WATERMARKOUTPUTave == True and WATERMARKOUTPUT2 == True: 
    WATERMARKOUTPUTFINAL = True
elif WATERMARKOUTPUT1 == True and WATERMARKOUTPUTave == True and WATERMARKOUTPUT2 == False: 
    WATERMARKOUTPUTFINAL = True
elif WATERMARKOUTPUT1 == False and WATERMARKOUTPUTave == True and WATERMARKOUTPUT2 == True: 
    WATERMARKOUTPUTFINAL = True
elif WATERMARKOUTPUT1 == True and WATERMARKOUTPUTave == False and WATERMARKOUTPUT2 == True: 
    WATERMARKOUTPUTFINAL = True


# SSIM DIFFERENCE

In [ ]:
img1 = CLEANARRAYFORMAINIMAGEFILEPNG
img2 = CLEANARRAYFORCOMPAREIMAGEFILEPNG

imageA = CLEANARRAYFORMAINIMAGEFILEPNG
imageB = CLEANARRAYFORCOMPAREIMAGEFILEPNG

grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

bigH, bigW,other = imageA.shape

In [ ]:
(score, diff) = compare_ssim(grayA, grayB, full=True)
diff = (diff * 255).astype('uint8')

In [ ]:
thresh = cv2.threshold(diff, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
areaDIFFERENCE = 0
for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    cv2.rectangle(imageA, (x, y), (x + w, y + h), (0, 0, 255), 2)
    cv2.rectangle(imageB, (x, y), (x + w, y + h), (0, 0, 255), 2)
    areaDIFFERENCE += (w*h)
perdiff =  ((areaDIFFERENCE/(bigW*bigH))*100)

# Compare Images

## Display All main points of connectors

In [ ]:
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

#keypoints
sift = cv2.xfeatures2d.SIFT_create()
keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
keypoints_2, descriptors_2 = sift.detectAndCompute(img2,None)
NUMBER_OF_KEYPOINST_IMG1 = len(keypoints_1)
NUMBER_OF_KEYPOINST_IMG2 = len(keypoints_2)

img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

## FINAL RUN

In [ ]:
keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
keypoints_2, descriptors_2 = sift.detectAndCompute(img2,None)
bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)
matches = bf.match(descriptors_1,descriptors_2)
matches = sorted(matches, key = lambda x:x.distance)
totalMATCHES = (len(matches))

In [ ]:
if img1 is None or img2 is None:
    exit(0)

minHessian = 400
detector = cv2.xfeatures2d_SURF.create(hessianThreshold=minHessian)
keypoints1, descriptors1 = detector.detectAndCompute(img1, None)
keypoints2, descriptors2 = detector.detectAndCompute(img2, None)
matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
knn_matches = matcher.knnMatch(descriptors1, descriptors2, 2)
ratio_thresh = 0.64
good_matches = []
for m,n in knn_matches:
    if m.distance < ratio_thresh * n.distance:
        good_matches.append(m)
img_matches = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], 3), dtype=np.uint8)
img3 = cv2.drawMatches(img1, keypoints1, img2, keypoints2, good_matches, img_matches, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
im = Image.fromarray(img3)
im.save('FINALWORKINGIMAGEPT1.png')
NUMEBROFMATCHESRATIOTEST = len(good_matches)

In [ ]:
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2, k=2)
good = []
for m,n in matches:
    if m.distance < 87:
        good.append([m])
    if n.distance < 87:
        good.append([n])
img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,None,flags =2)

im = Image.fromarray(img3)
im.save('FINALWORKINGIMAGEPT2.png')
NUMEBROFMATCHESDISTANCETEST = len(good)

##  Output

In [ ]:
print("**************************")
print(MAINIMAGEFILEPNG)
print(COMPAREIMAGEFILEPNG)
print("**************************")
print("IMAGE 1 is FAKE")
print("IMAGE 2 is REAL")
print("**************************")
print("Shape of image 1 is ")
print(x11, " ", y11, " ", z11)
print("Shape of image 2 is")
print(x2, " ", y2, " ", z2)
print("**************************")
print("The SSIM score is ")
if score > 0.25:
    SSIMFINALSCORE = True
else:
    SSIMFINALSCORE = False
print(score)
print("**************************")
print("The Percent Difference is")
print(perdiff)
if perdiff > 0.8:
    PERCENTDIFFERNCE = True
else:
    PERCENTDIFFERNCE = False
print("**************************")
print("The number of key points in image 1 is")
print(NUMBER_OF_KEYPOINST_IMG1)
print("The number of key points in image 2 is")
print(NUMBER_OF_KEYPOINST_IMG2)
print("**************************")
print("The total number of matches is")
print(totalMATCHES)
print("**************************")
print("Number of matches for the distance test is", NUMEBROFMATCHESDISTANCETEST)
print("Number of matches for the ratio test is", NUMEBROFMATCHESRATIOTEST)

# FINAL OUTPUT

In [ ]:
print("BASIC INFO:")
print(MAINIMAGEFILEPNG)
print(COMPAREIMAGEFILEPNG)
print("************************************************************************")
print("Shape of image 1 is ")
print(x11, " ", y11, " ", z11)
print("Shape of image 2 is")
print(x2, " ", y2, " ", z2)
print("************************************************************************")

In [ ]:
#display(Markdown('**some markdown** '))
print("Barcode Output:")
print(BARCODEISCORRECT)
print("************************************************************************")
print("DATE OUTPUT:")
print(DATEWORKING, " because ", WHYDATEISWORKING)
print("************************************************************************")
print("TITLE OUTPUT:")
print(TITLEIMAGECOMPARE)
print("Difference in BLUE       Difference in GREEN     Difference in RED")
print(PERCENT_BLUE_DIFFERENCE_TITLE, "\t\t\t", PERCENT_GREEN_DIFFERENCE_TITLE, "\t\t\t", PERCENT_RED_DIFFERENCE_TITLE)
print("************************************************************************")
print("TEXT OUTPUT:")
print(SIMILARTYBETWEENTEXTINDOCUMENT)
print("************************************************************************")
print("WATER MARK OUTPUT:")
print(WATERMARKOUTPUTFINAL)
print("************************************************************************")
print("SSIM SCORE OUTPUT")
print(SSIMFINALSCORE)
print("************************************************************************")
print("Percent Difference OUTPUT")
print(PERCENTDIFFERNCE)
print("************************************************************************")

In [ ]:
countREAL = 0
countFAKE = 0
if BARCODEISCORRECT:
    countREAL += 1
else:
    countFAKE += 1
#------------------------
if DATEWORKING:
    countREAL += 1
else:
    countFAKE += 1
#------------------------
if TITLEIMAGECOMPARE:
    countREAL += 1
else:
    countFAKE += 1
#------------------------
if SIMILARTYBETWEENTEXTINDOCUMENT:
    countREAL += 1
else:
    countFAKE += 1
#------------------------
if WATERMARKOUTPUTFINAL:
    countREAL += 1
else:
    countFAKE += 1
#------------------------
if SSIMFINALSCORE:
    countREAL += 1
else:
    countFAKE += 1
#------------------------
if PERCENTDIFFERNCE:
    countREAL += 1
else:
    countFAKE += 1
print(countREAL, " ", countFAKE)

In [ ]:
documentanswer = False
if countREAL >= 4:
    documentanswer = True

In [ ]:
if documentanswer:
    display(Markdown('**Document is REAL**'))
else:
    display(Markdown('**Document is FAKE**'))